# Cleaning the Data

In [37]:
# import libraries needed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
np.random.seed(0)

In [19]:
# load original data into dataframe and check shape
df_ori = pd.read_csv('dataset_diabetes/diabetic_data.csv')
#print(df_ori.shape)
df = df_ori.copy(deep=True)

In [20]:
df['readmitted'] = df['readmitted'].replace('>30', 0)
df['readmitted'] = df['readmitted'].replace('<30', 1) #should we code it into 1 and 2?
df['readmitted'] = df['readmitted'].replace('NO', 0)

In [21]:
df.drop(columns = ['patient_nbr','citoglipton','weight','examide','encounter_id', 'medical_specialty', 'payer_code'],inplace=True)
#df.drop(columns = ['citoglipton','weight','examide','encounter_id', 'medical_specialty', 'payer_code'],inplace=True)

#df = df.drop_duplicates(subset= ['patient_nbr'], keep = 'first')
drop_Idx = set()
drop_Idx = drop_Idx.union(set(df[df['discharge_disposition_id'] == 11].index))
drop_Idx = drop_Idx.union(set(df['gender'][df['gender'] == 'Unknown/Invalid'].index))
drop_Idx = drop_Idx.union(set(df['diag_1'][df['diag_1']=='?'].index))
drop_Idx = drop_Idx.union(set(df['diag_2'][df['diag_2']=='?'].index))
drop_Idx = drop_Idx.union(set(df['diag_3'][df['diag_3']=='?'].index))
new_Idx = list(set(df.index) - set(drop_Idx))
df = df.iloc[new_Idx]
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
1,Caucasian,Female,[10-20),1,1,7,3,59,0,18,...,No,Up,No,No,No,No,No,Ch,Yes,0
2,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,...,No,No,No,No,No,No,No,No,Yes,0
3,Caucasian,Male,[30-40),1,1,7,2,44,1,16,...,No,Up,No,No,No,No,No,Ch,Yes,0
4,Caucasian,Male,[40-50),1,1,7,1,51,0,8,...,No,Steady,No,No,No,No,No,Ch,Yes,0
5,Caucasian,Male,[50-60),2,1,2,3,31,6,16,...,No,Steady,No,No,No,No,No,No,Yes,0


In [22]:
keys = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', 'glyburide', 
        'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'insulin', 'glyburide-metformin', 'tolazamide', 
        'metformin-pioglitazone','metformin-rosiglitazone', 'glimepiride-pioglitazone', 
        'glipizide-metformin', 'troglitazone', 'tolbutamide', 'acetohexamide']

for col in keys:
    df[col] = df[col].replace('No', 0)
    df[col] = df[col].replace('Steady', 1)
    df[col] = df[col].replace('Up', 1)
    df[col] = df[col].replace('Down', 1)
    
df['change'] = df['change'].replace('Ch', 1)
df['change'] = df['change'].replace('No', 0)

df['gender'] = df['gender'].replace('Male', 1)
df['gender'] = df['gender'].replace('Female', 0)

df['diabetesMed'] = df['diabetesMed'].replace('Yes', 1)
df['diabetesMed'] = df['diabetesMed'].replace('No', 0)

df['age'] = df['age'].replace('[0-10)', 5)
df['age'] = df['age'].replace('[10-20)', 15)
df['age'] = df['age'].replace('[20-30)', 25)
df['age'] = df['age'].replace('[30-40)', 35)
df['age'] = df['age'].replace('[40-50)', 45)
df['age'] = df['age'].replace('[50-60)', 55)
df['age'] = df['age'].replace('[60-70)', 65)
df['age'] = df['age'].replace('[70-80)', 75)
df['age'] = df['age'].replace('[80-90)', 85)
df['age'] = df['age'].replace('[90-100)', 95)

df['race'] = df['race'].replace('Caucasian', 0)
df['race'] = df['race'].replace('AfricanAmerican', 1)
df['race'] = df['race'].replace('Hispanic', 2)
df['race'] = df['race'].replace('Asian', 3)
df['race'] = df['race'].replace('Other', 4)

df = df.replace('?', -1)

In [23]:
diags = ['diag_1','diag_2','diag_3']
    
def detection(value):
    if value[0]== "V" or value[0] == 'E':
        value = '0'
        return value # from new york united healthcare
    else:
        return value
    
for f in diags:
    for i in df[f].index:
        df[f].at[i] = detection(df[f].at[i])
        
for i in df['diag_1']:
    b = 'V' in str(i)
    c = 'E' in str(i)
    if b or c:
        print(str(i))

In [24]:
df['service_utilization'] = df['number_outpatient'] + df['number_emergency'] + df['number_inpatient']
print("geelo ", df.columns)
df.drop(['number_outpatient', 'number_emergency', 'number_inpatient'], axis=1, inplace=True)
df.columns

geelo  Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'service_utilization'],
      dtype='object')


Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'service_utilization'],
      dtype='object')

In [25]:
df_max_glu_serum = pd.get_dummies(df['max_glu_serum'])
df_A1Cresult = pd.get_dummies(df['A1Cresult'])
df_insulin = pd.get_dummies(df['insulin'])
df_discharge_disposition_id = pd.get_dummies(df['discharge_disposition_id'])
df_admission_source_id = pd.get_dummies(df['admission_source_id'])
df_admission_type_id = pd.get_dummies(df['admission_type_id'])

      
df = pd.concat([df,df_max_glu_serum, df_A1Cresult, 
                df_insulin, df_discharge_disposition_id, 
                df_admission_source_id, df_admission_type_id], axis=1)
df.drop([ 'max_glu_serum', 'A1Cresult', 'insulin','discharge_disposition_id', 'admission_source_id', 
                  'admission_type_id'], axis=1, inplace=True)


In [26]:
from sklearn.model_selection import train_test_split

feature_col_names = df.columns
x = df[feature_col_names]
x = df.loc[:, df.columns != 'readmitted']
y = df['readmitted']
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.20, random_state=1)

In [27]:
from imblearn.over_sampling import SMOTE
from collections import Counter
print('Original dataset shape {}'.format(Counter(y)))
smt = SMOTE(random_state=20)
train_input_new, train_output_new = smt.fit_sample(x.as_matrix(), y)
print('New dataset shape {}'.format(Counter(train_output_new)))
train_input_new = pd.DataFrame(train_input_new, columns = list(x.columns))
Xtr, Xva, Ytr, Yva = train_test_split(train_input_new, train_output_new, test_size=0.20, random_state=0)

Original dataset shape Counter({0: 87351, 1: 11250})


C:\Users\Lillian\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


New dataset shape Counter({0: 87351, 1: 87351})


# Baseline Neural Network

In [51]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier()
clf.fit(Xtr, Ytr)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [52]:
y_train_preds = clf.predict(Xtr)
y_valid_preds = clf.predict(Xva)

print("Training Accuracy Score:  ", accuracy_score(Ytr, y_train_preds ))
print("Validation Accuracy Score:  ", accuracy_score(Yva, y_valid_preds ))

Training Accuracy Score:   0.52810869985189
Validation Accuracy Score:   0.5225666122892877


# Finding optimal number of layers

In [61]:
layers = np.array([1, 50, 100, 200, 300])

for l in layers:
    nn = MLPClassifier(hidden_layer_sizes = (l,), max_iter = 400)
    nn.fit(Xtr, Ytr)

    y_train_preds = nn.predict(Xtr)
    y_valid_preds = nn.predict(Xva)
    
    print(l, " layers")
    print("Training Accuracy Score:  ", accuracy_score(Ytr, y_train_preds ))
    print("Validation Accuracy Score:  ", accuracy_score(Yva, y_valid_preds ))
    print("")

1  layers
Training Accuracy Score:   0.5006904644357153
Validation Accuracy Score:   0.49718096219341174

50  layers
Training Accuracy Score:   0.6136833594493457
Validation Accuracy Score:   0.611945851578375

100  layers
Training Accuracy Score:   0.6370518241855739
Validation Accuracy Score:   0.6316075670415844

200  layers
Training Accuracy Score:   0.7835948512102804
Validation Accuracy Score:   0.7803440084714233

300  layers
Training Accuracy Score:   0.6564706892480735
Validation Accuracy Score:   0.6506682693683639



# Testing different activation functions

In [64]:
functions = np.array(["identity", "logistic", "tanh", "relu"])
for f in functions:

    nn = MLPClassifier(hidden_layer_sizes = (200,), activation = f)
    nn.fit(Xtr, Ytr)

    y_train_preds = nn.predict(Xtr)
    y_valid_preds = nn.predict(Xva)

    print("")
    print(f)
    print("Training Accuracy Score:  ", accuracy_score(Ytr, y_train_preds ))
    print("Validation Accuracy Score:  ", accuracy_score(Yva, y_valid_preds ))


identity
Training Accuracy Score:   0.5169467877304827
Validation Accuracy Score:   0.5130076414527346

logistic
Training Accuracy Score:   0.7651848512818311
Validation Accuracy Score:   0.7608826307203572

tanh
Training Accuracy Score:   0.6741150964861442
Validation Accuracy Score:   0.6763401161958731

relu
Training Accuracy Score:   0.663990669786278
Validation Accuracy Score:   0.6633467845797201


# Optimized Neural Network parameters

In [83]:
nn = MLPClassifier(hidden_layer_sizes = (200,), activation = "logistic")
nn.fit(Xtr, Ytr)

y_train_preds = nn.predict(Xtr)
y_valid_preds = nn.predict(Xva)

print("Training Accuracy Score:  ", accuracy_score(Ytr, y_train_preds ))
print("Validation Accuracy Score:  ", accuracy_score(Yva, y_valid_preds ))

Training Accuracy Score:   0.7755954808566052
Validation Accuracy Score:   0.7727311754099768


# 10-fold Cross Validation

In [84]:
from sklearn.model_selection import cross_val_score
results = cross_val_score(nn, x, y, cv=10)
avg = results.mean()

In [85]:
print(avg)

0.8860254967990155


In [86]:
nn1 = MLPClassifier(hidden_layer_sizes = (100,), activation = "relu")
nn1.fit(Xtr, Ytr)

y_train_preds = nn1.predict(Xtr)
y_valid_preds = nn1.predict(Xva)

print("Training Accuracy Score:  ", accuracy_score(Ytr, y_train_preds ))
print("Validation Accuracy Score:  ", accuracy_score(Yva, y_valid_preds ))

Training Accuracy Score:   0.6070148324639921
Validation Accuracy Score:   0.598008070747832


In [87]:
from sklearn.model_selection import cross_val_score
results = cross_val_score(nn, x, y, cv=10)
avg = results.mean()
print(avg)

0.8859240769207192


# Conclusion
After testing these parameters, it is found that the optimal amount of hidden layers is 200 and the optimal activation function is the logistic (sigmoid) function.